# Judicial Vacancies Data Source Exploration

This notebook demonstrates how to use the `dataset` module to fetch and process judicial vacancy data.

## Overview

We'll:
1. Import the necessary modules
2. Fetch HTML data from the judicial vacancies archive
3. Extract vacancy data from the HTML
4. Convert the data to a pandas DataFrame
5. Save the raw data to a CSV file

## Setup

In [10]:
import sys
!{sys.executable} -m pip list | grep nomination_predictor

nomination_predictor      0.0.1             /home/wsl2ubuntuuser/nomination_predictor


In [11]:
%load_ext autoreload
%autoreload 2

import sys
import os
from pathlib import Path
import pandas as pd
import typer
from datetime import datetime

# Import our data processing module
from nomination_predictor import dataset
from nomination_predictor.config import RAW_DATA_DIR

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# Set up paths
PROJECT_ROOT = Path.cwd().parent
DATA_RAW = PROJECT_ROOT / 'data' / 'raw'
DATA_RAW.mkdir(parents=True, exist_ok=True)

In [ ]:
# Verify the data directory exists and is writable
if not DATA_RAW.exists():
    print(f"Error: Data directory does not exist: {DATA_RAW}")
elif not os.access(DATA_RAW, os.W_OK):
    print(f"Error: No write permission for directory: {DATA_RAW}")
else:
    print(f"Data directory is ready: {DATA_RAW}")

Data directory is ready: /home/wsl2ubuntuuser/nomination_predictor/data/raw


## 1. Fetch and Process Data

Let's fetch the data for the range of available years and process it.

In [ ]:
def load_judicial_data(years_back=15, force_refresh=False):
    """
    Load judicial data using the dataset module.
    
    Args:
        years_back: Number of years of historical data to fetch
        force_refresh: If True, force refetching data even if files exist
        
    Returns:
        Tuple of (vacancies_df, confirmations_df, emergencies_df, combined_df)
    """
    output_file = DATA_RAW / "judicial_data.csv"
    
    # Only run the pipeline if output doesn't exist or force_refresh is True
    if force_refresh or not output_file.exists():
        print("Running data pipeline...")
        dataset.main(
            output_dir=DATA_RAW,
            output_filename=output_file.name,
            years_back=years_back
        )
    
    # Load individual datasets
    vacancies_path = DATA_RAW / "judicial_vacancies.csv"
    confirmations_path = DATA_RAW / "judicial_confirmations.csv"
    emergencies_path = DATA_RAW / "judicial_emergencies.csv"
    
    # Read the data
    combined_df = pd.read_csv(output_file, sep='|') if output_file.exists() else None
    vacancies_df = pd.read_csv(vacancies_path, sep='|') if vacancies_path.exists() else None
    confirmations_df = pd.read_csv(confirmations_path, sep='|') if confirmations_path.exists() else None
    emergencies_df = pd.read_csv(emergencies_path, sep='|') if emergencies_path.exists() else None
    
    return vacancies_df, confirmations_df, emergencies_df, combined_df

In [20]:
print("Loading judicial data...")
vacancies_df, confirmations_df, emergencies_df, combined_df = load_judicial_data(
    years_back=2,  # Adjust as needed
    force_refresh=False  # Set to True to refetch data
)

Loading judicial data...


In [ ]:
print("\nData Summary:")
print(f"Vacancies: {len(vacancies_df) if vacancies_df is not None else 0} records")
print(f"Confirmations: {len(confirmations_df) if confirmations_df is not None else 0} records")
print(f"Emergencies: {len(emergencies_df) if emergencies_df is not None else 0} records")
print(f"Combined: {len(combined_df) if combined_df is not None else 0} records")


Data Summary:
Vacancies: 891 records
Confirmations: 1245 records
Emergencies: 353 records
Combined: 2489 records


In [ ]:
if combined_df is not None:
    print("\nSample data from combined dataset:")
    display(combined_df.head())
    
    # Basic statistics
    print("\nBasic Statistics:")
    
    if 'vacancy_date' in combined_df.columns:
        print("\nDate Range:")
        print(f"Earliest vacancy: {combined_df['vacancy_date'].min()}")
        print(f"Latest vacancy: {combined_df['vacancy_date'].max()}")
    
    print('\nRecords per year:')
    print(combined_df['source_year'].value_counts().sort_index())
    
    if 'circuit_district' in combined_df.columns:
        print("\nRecords by Circuit/District:")
        print(combined_df['circuit_district'].value_counts().head(10))
        


Sample data from combined dataset:


,court,incumbent,vacancy_reason,vacancy_date,nominee,nomination_date,source_year,source_month,source_page_type,confirmation_date,circuit_district,vacancy_created_by,reason,days_pending,weighted,adjusted
0,01 - CCA,"Howard,Jeffrey R.",Senior,03/31/2022,"Aframe,Seth Robert",10/04/2023,2024,1,vacancies,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,02 - CT,"Merriam,Sarah A. L.",Elevated,09/15/2022,"Russell,Sarah French",10/04/2023,2024,1,vacancies,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,02 - NY-S,"Gardephe,Paul G.",Senior,08/09/2023,NaN,NaN,2024,1,vacancies,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,02 - NY-W,"Geraci Jr.,Frank Paul",Senior,04/01/2023,"Holland,Colleen Danielle",09/11/2023,2024,1,vacancies,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,03 - CCA,"Greenaway Jr.,Joseph A.",Retired,06/15/2023,"Mangi,Adeel Abdullah",11/27/2023,2024,1,vacancies,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Basic Statistics:

Date Range:
Earliest vacancy: 01/01/2018
Latest vacancy: 12/31/2024

Records per year:
source_year
2024    2013
2025     476
Name: count, dtype: int64

Records by Circuit/District:
circuit_district
07 - ILN     66
02 - NYE     55
09 - CAC     54
02 - NYS     51
03 - NJ      46
09 - CAN     46
06 - MIE     44
11 - FL-M    42
05 - TX-W    40
01 - MA      37
Name: count, dtype: int64


## 2. Save Raw Data

Save the raw data to a CSV file in the `data/raw` directory.

In [ ]:
def save_raw_data(df, year):
    """Save the raw data to a CSV file."""
    if df is None or df.empty:
        print('No data to save.')
        return
    
    filename = DATA_RAW / f'judicial_vacancies_{year}.csv'
    try:
        dataset.save_to_csv(df, filename)
        print(f'Data saved to {filename}')
    except Exception as e:
        print(f'Error saving data: {e}')

# Save the data we just fetched
if df is not None:
    save_raw_data(df, year_to_fetch)

NameError: name 'df' is not defined

## 3. Load and Explore the Saved Data

Let's verify that we can load the saved data.

In [ ]:
def load_raw_data(year):
    """Load raw data from a CSV file."""
    filename = DATA_RAW / f'judicial_vacancies_{year}.csv'
    if not filename.exists():
        print(f'File not found: {filename}')
        return None
    
    try:
        df = pd.read_csv(filename)
        print(f'Loaded {len(df)} records from {filename}')
        return df
    except Exception as e:
        print(f'Error loading {filename}: {e}')
        return None

# Load the data we just saved
loaded_df = load_raw_data(year_to_fetch)
if loaded_df is not None:
    print('\nDataFrame info:')
    display(loaded_df.info())
    print('\nFirst few records:')
    display(loaded_df.head())

Loaded 2380 records from /home/wsl2ubuntuuser/nomination_predictor/data/raw/judicial_vacancies_2024.csv

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2380 entries, 0 to 2379
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   court                 2380 non-null   object 
 1   vacancy_date          2182 non-null   object 
 2   status                0 non-null      float64
 3   nominating_president  0 non-null      float64
 4   nominee               1543 non-null   object 
 5   year                  2380 non-null   int64  
 6   month                 2380 non-null   object 
dtypes: float64(2), int64(1), object(4)
memory usage: 130.3+ KB


None


First few records:


,court,vacancy_date,status,nominating_president,nominee,year,month
0,US Court of Appeals,NaN,NaN,NaN,NaN,2024,vacancy summary for december 2024
1,US District Courts (includes territorial courts*),NaN,NaN,NaN,NaN,2024,vacancy summary for december 2024
2,US Court of International Trade,NaN,NaN,NaN,NaN,2024,vacancy summary for december 2024
3,US Court of Federal Claims,NaN,NaN,NaN,NaN,2024,vacancy summary for december 2024
4,US Supreme Court,NaN,NaN,NaN,NaN,2024,vacancy summary for december 2024


## Next Steps

1. **Data Cleaning**: In the next notebook, we'll clean and preprocess this data.
2. **Exploratory Analysis**: We'll explore the data to understand its structure and quality.
3. **Feature Engineering**: We'll create additional features that might be useful for analysis.
4. **Visualization**: We'll create visualizations to understand trends and patterns.